<a href="https://colab.research.google.com/github/TobiPrae/customer_segmentation/blob/main/CustomerSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline
print("Libraries successfully imported")

import gc

# machine learning magic
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Libraries successfully imported


In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("Everything successfully set up")

Everything successfully set up


# Load Data

In [3]:
# Get list of every available file in google drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list[:1]:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: meta_data.xlsx, id: 1QwV131IP74EHaBrKAnlcGk4EaMwBsTHE


In [4]:
# download demographic data
download_data = drive.CreateFile({'id': '135UcsbUyLcwS16QHa7lFcXHKhuy7HwWF'})
download_data.GetContentFile('azdias.csv')  
df_azdias = pd.read_csv('azdias.csv', engine="python", sep=';')

print(df_azdias.shape)

(891221, 366)


In [5]:
# download customer data
download_data = drive.CreateFile({'id': '1eDq76GDZLXeonowxPrbwF-EHZwvaENLK'})
download_data.GetContentFile('customers.csv')  
df_customers = pd.read_csv('customers.csv', engine="python", sep=';')

print(df_customers.shape)

(191652, 369)


In [6]:
# download attribute values and clean them
download_data = drive.CreateFile({'id': '1wFNhCQm7Cv9CGJ7Pm_RbtrJdnLb4zyuC'})
download_data.GetContentFile('attributes_values.xlsx')  
df_attributes_values = pd.read_excel('attributes_values.xlsx', names=["None", "attribute", "description", "value", "meaning"])
df_attributes_values = df_attributes_values.drop(columns=['None'])[1:]
# Forward filling the attribute values df for better filtering/slicing
df_attributes_values["attribute"] = df_attributes_values["attribute"].fillna(method="ffill")
df_attributes_values["description"] = df_attributes_values["description"].fillna(method="ffill")
print(df_attributes_values.shape)

(2258, 4)


In [7]:
# download attribute information levels and clean them
download_data = drive.CreateFile({'id': '1icPBxHTV0zysKvQSJhSDPDhyH8Qd3uqW'})
download_data.GetContentFile('information_levels_attributes.xlsx')  
df_information_levels = pd.read_excel('information_levels_attributes.xlsx', names=["None", "Information_Level", "attribute", "description", "additional_notes"])
df_information_levels = df_information_levels.drop(columns=["None"])[1:]
print(df_information_levels.shape)

(313, 4)


In [8]:
# download demographic data
download_data = drive.CreateFile({'id': '1QwV131IP74EHaBrKAnlcGk4EaMwBsTHE'})
download_data.GetContentFile('meta_data.xlsx')  
meta_data = pd.read_excel('meta_data.xlsx')
meta_data = meta_data.iloc[:365, :]
print(meta_data.shape)

(365, 16)


# Functions

In [9]:
def get_description(col):
  '''
  Gets description for given column.

  Args:
  - col: column name

  Returns:
  - description or "No description available."
  '''
  if(df_attributes_values.loc[df_attributes_values["attribute"] == col].shape[0] == 0):
    return "No description available."
  else:
    return df_attributes_values.loc[df_attributes_values["attribute"] == col]["description"].values[0]

In [10]:
def transform_PRAEGENDE_JUGENDJAHRE(x):
  '''
    Transform value for feature PRAEGENDE_JUGENDJAHRE.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(x in [1, 2]):
    return 40
  elif(x in [3, 4]):
    return 50
  elif(x in [5, 6, 7]):
    return 60
  elif(x in [8, 9]):
    return 70
  elif(x in [10, 11, 12, 13]):
    return 80
  elif(x in [14, 15]):
    return 90

In [11]:
def transform_CAMEO_INTL_2015(x):
  '''
  Transform value for feature CAMEO_INTL_2015.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(str(x)[0] == '1'):
    return '1'
  if(str(x)[0] == '2'):
    return '2'
  if(str(x)[0] == '3'):
    return '3'
  if(str(x)[0] == '4'):
    return '4'
  if(str(x)[0] == '5'):
    return '5'
  else:
    return np.nan

In [12]:
def transform_CAMEO_DEUG_2015(x):
  '''
  Transform value for feature CAMEO_DEUG_2015.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(x == 'X'):
    return np.nan
  else:
    return x

In [13]:
def create_meta_dataframa(df):
  '''
  Creates data frame with meta information.

  Args:
  - Nothing

  Returns:
  - meta_df
  '''
  meta_df = pd.DataFrame(df.columns.tolist())
  meta_df = meta_df.rename(columns={0: "column_name"})
  meta_df["na_percentage"] = meta_df["column_name"].apply(lambda x: round(df[x].isna().sum()/df.shape[0], 4))
  meta_df["dtypes"] =  meta_df["column_name"].apply(lambda x: set(df[x].map(type).tolist()))
  meta_df["n_dtypes"] = meta_df["column_name"].apply(lambda x: len(set(df[x].map(type).tolist())))
  meta_df["unique_valuess"] = meta_df["column_name"].apply(lambda x: len(df[x].value_counts()))
  return meta_df

In [14]:
def transformation_pipeline(df, column_threshold = 0.3, row_threshold = 0.3):

  print(f"shape before: {df.shape}")

  # Drop ID column
  df = df.drop(columns="LNR")
  print("index droped")

  # Drop columns with nan percentage over certain threshold
  meta_df = pd.DataFrame(df.columns.tolist())
  meta_df = meta_df.rename(columns={0: "column_name"})
  meta_df["na_percentage"] = meta_df["column_name"].apply(lambda x: df[x].isna().sum()/df.shape[0])
  drop_columns = meta_df.loc[meta_df["na_percentage"] > column_threshold]["column_name"].tolist()
  df = df.drop(columns=drop_columns)
  print(f"columns with nan threshold of {column_threshold} droped: ")
  print(drop_columns)

  # Drop rows with nan percentage over certain threshold
  df["row_nan_percentage"] = pd.Series(df.index).apply(lambda x: df.iloc[x, :].isna().sum()/df.shape[1])
  df = df.loc[df["row_nan_percentage"] < row_threshold]
  print(f"rows with nan threshold of {row_threshold} droped")  

  ### Drop columns with unknown data types and no description ###

  unknown_cols = meta_data.loc[meta_data["type"] == "Unknown"]["column_name"].tolist()
  df_transformed = df.drop(columns=list(set(unknown_cols).intersection(set(df.columns))))
  print("columns with unknown datatype droped")

  ### Transformation of binary columns ###

  df_transformed["ANREDE_KZ"] = df_transformed["ANREDE_KZ"].replace(2, 0) 
  df_transformed["GREEN_AVANTGARDE"] = df_transformed["GREEN_AVANTGARDE"].fillna(df_transformed["GREEN_AVANTGARDE"].mode().values[0])
  df_transformed["OST_WEST_KZ"] = df_transformed["OST_WEST_KZ"].apply(lambda x: 1 if x == 'W' else (0 if 'O' else x))
  df_transformed["SOHO_KZ"] = df_transformed["SOHO_KZ"].fillna(df_transformed["SOHO_KZ"].mode().values[0])
  df_transformed["ANREDE_KZ"] = df_transformed["ANREDE_KZ"].fillna(df_transformed["ANREDE_KZ"].mode().values[0])
  df_transformed["OST_WEST_KZ"] = df_transformed["OST_WEST_KZ"].fillna(df_transformed["OST_WEST_KZ"].mode().values[0])
  print("binary columns transformed")

  ### Transformation of numerical columns ###

  numerical_columns = meta_data.loc[meta_data["type"] == "Numerical"]["column_name"].tolist()
  for col in list(set(numerical_columns).intersection(set(df_transformed.columns))):
    df_transformed[col] = df_transformed[col].fillna(df_transformed[col].median())
  print("numeric columns transformed")

  ### Transformation of ordinal columns ###

  # Transform ordinal columns that have no value or -1 for unknown
  ordinal_no_unknown_cols = meta_data.loc[(meta_data["9 unknown"] != 'x') & (meta_data["0  unknown"] != 'x') & (meta_data["type"] == "Ordinal")]["column_name"].tolist()
  for col in ordinal_no_unknown_cols:
    df_transformed[col] = df_transformed[col].fillna(df_transformed[col].median())
  print("ordinal columns that have no value for unkown transformed")

  # Transform ordinal colums that have 9 or -1 values for unknow
  ordinal_9_1_cols = meta_data.loc[(meta_data["9 unknown"] == 'x') & (meta_data["type"] == "Ordinal")]["column_name"].tolist()
  for col in ordinal_9_1_cols:
    df_transformed[col] = df_transformed[col].replace(9, np.nan).fillna(df_transformed[col].median())
  print("ordinal colums that have 9 or -1 values for unknow transformed")

  # Transform ordinal columns that have 0 or -1 values for unknown
  ordinal_0_1_cols = meta_data.loc[(meta_data["0  unknown"] == 'x') & (meta_data["type"] == "Ordinal")]["column_name"].tolist()
  for col in ordinal_0_1_cols:
    df_transformed[col] = df_transformed[col].replace(0, np.nan).fillna(df_transformed[col].median())
  print("ordinal colums that have 0 or -1 values for unknow transformed")

  ### Transformation of categorical features ###

  # Transform categorical columns that have 0 values for unknown
  categorical_0_columns = meta_data.loc[(meta_data["9 unknown"] != 'x') & (meta_data["0  unknown"] == 'x') & (meta_data["type"] == "Categorical")]["column_name"].tolist()
  for col in categorical_0_columns:
    df_transformed[col] = df_transformed[col].replace({-1: np.nan, 0: np.nan})
  print("categorical columns that have 0 values for unknown transformed")

  # Transform "PRAEGENDE_JUGENDJAHRE"
  df_transformed["PRAEGENDE_JUGENDJAHRE"] = df_transformed["PRAEGENDE_JUGENDJAHRE"].apply(transform_PRAEGENDE_JUGENDJAHRE)
  print('"PRAEGENDE_JUGENDJAHRE" transformed')

  # Transform "ZABEOTYP"
  df_transformed["ZABEOTYP"] = df_transformed["ZABEOTYP"].replace({9: np.nan})
  print('"ZABEOTYP" transformed')

  # Transform "D19_KONSUMTYP", "FINANZTYP", "GEBAEUDETYP", "GFK_URLAUBERTYP", "SHOPPER_TYP"
  cat_temp = ["D19_KONSUMTYP", "FINANZTYP", "GEBAEUDETYP", "GFK_URLAUBERTYP", "SHOPPER_TYP"]
  for col in cat_temp:
    df_transformed[col] = df_transformed[col].replace({-1: np.nan})
  print('"D19_KONSUMTYP", "FINANZTYP", "GEBAEUDETYP", "GFK_URLAUBERTYP", "SHOPPER_TYP" transformed')

  # Drop "CAMEO_DEU_2015" and "AGER_TYP"
  df_transformed = df_transformed.drop(columns=["CAMEO_DEU_2015", "AGER_TYP"])
  print('"CAMEO_DEU_2015" and "AGER_TYP" droped')

  # Transform "CAMEO_INTL_2015"
  df_transformed["CAMEO_INTL_2015"] = df_transformed["CAMEO_INTL_2015"].apply(transform_CAMEO_INTL_2015)
  print("CAMEO_INTL_2015 transformed")

  # Transform "CAMEO_DEUG_2015"
  df_transformed["CAMEO_DEUG_2015"] = df_transformed["CAMEO_DEUG_2015"].apply(transform_CAMEO_DEUG_2015)
  print("CAMEO_DEUG_2015 transformed")

  # Create dummies
  cat_cols = meta_data.loc[meta_data["type"] == "Categorical"]["column_name"].tolist()
  print(cat_cols)
  dummy_cols = list(set(cat_cols).intersection(set(df_transformed.columns)))
  print(dummy_cols)
  for col in dummy_cols:
    col_dum_df = pd.get_dummies(df_transformed[col], prefix=col, dummy_na=True)
    df_transformed = pd.concat([df_transformed, col_dum_df], axis=1)

  df_transformed = df_transformed.drop(columns=dummy_cols)
  print("Dummies created")

  # Transform everything to float
  df_transformed = df_transformed.astype(float)

  print(f"shape after: {df_transformed.shape}")
  return df_transformed



# Part 0: Clean and Standardize Data

In [15]:
df_azdias_transformed = transformation_pipeline(df_azdias)
#meta_df_azdias = create_meta_dataframa(df_azdias_transformed)

shape before: (891221, 366)
index droped
columns with nan threshold of 0.3 droped: 
['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992', 'KK_KUNDENTYP']
rows with nan threshold of 0.3 droped
columns with unknown datatype droped
binary columns transformed
numeric columns transformed
ordinal columns that have no value for unkown transformed
ordinal colums that have 9 or -1 values for unknow transformed
ordinal colums that have 0 or -1 values for unknow transformed
categorical columns that have 0 values for unknown transformed
"PRAEGENDE_JUGENDJAHRE" transformed
"ZABEOTYP" transformed
"D19_KONSUMTYP", "FINANZTYP", "GEBAEUDETYP", "GFK_URLAUBERTYP", "SHOPPER_TYP" transformed
"CAMEO_DEU_2015" and "AGER_TYP" droped
CAMEO_INTL_2015 transformed
CAMEO_DEUG_2015 transformed
['AGER_TYP', 'CAMEO_DEU_2015', 'CAMEO_DEUG_2015', 'CAMEO_INTL_2015', 'CJT_GESAMTTYP', 'D19_KONSUMTYP', 'FINANZTYP', 'GEBAEUDETYP', 'GFK_URLAUBERTYP', 'HEALTH_TYP', 'NATIONALITAET_KZ', 'PRAEGENDE_JUGENDJAHR

In [16]:
#df_customers_transformed = transformation_pipeline(df_customers)
#meta_df_customers = create_meta_dataframa(df_customers_transformed)

In [17]:
#print(set(df_azdias_transformed.columns).difference(set(df_customers_transformed.columns)))
#print(set(df_customers_transformed.columns).difference(set(df_azdias_transformed.columns)))

In [19]:
# Scale features
cols = df_azdias_transformed.columns.tolist()
scaled_features = StandardScaler().fit_transform(df_azdias_transformed.values)

# Free some memory
del df_azdias
del df_azdias_transformed
gc.collect()

# Create df
scaled_df = pd.DataFrame(scaled_features, columns=cols)

1048615

In [23]:
scaled_df.head(5)

,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,BALLRAUM,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,D19_DIGIT_SERV,D19_DROGERIEARTIKEL,D19_ENERGIE,D19_FREIZEIT,D19_GARTEN,D19_GESAMT_ANZ_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_OFFLINE_DATUM,D19_GESAMT_ONLINE_DATUM,D19_GESAMT_ONLINE_QUOTE_12,D19_HANDWERK,D19_HAUS_DEKO,D19_KINDERARTIKEL,D19_KOSMETIK,D19_LEBENSMITTEL,D19_LOTTO,D19_NAHRUNGSERGAENZUNG,D19_RATGEBER,D19_REISEN,D19_SAMMELARTIKEL,D19_SCHUHE,D19_SONSTIGE,D19_TECHNIK,D19_TELKO_ANZ_12,D19_TELKO_ANZ_24,D19_TELKO_DATUM,D19_TELKO_MOBILE,D19_TELKO_OFFLINE_DATUM,D19_TELKO_ONLINE_DATUM,D19_TELKO_REST,D19_TIERARTIKEL,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,D19_VERSAND_DATUM,D19_VERSAND_OFFLINE_DATUM,D19_VERSAND_ONLINE_DATUM,D19_VERSAND_ONLINE_QUOTE_12,D19_VERSAND_REST,D19_VERSI_ANZ_12,D19_VERSI_ANZ_24,D19_VERSICHERUNGEN,D19_VOLLSORTIMENT,D19_WEIN_FEINKOST,EWDICHTE,FINANZ_ANLEGER,FINANZ_HAUSBAUER,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_UNAUFFAELLIGER,FINANZ_VORSORGER,GEBAEUDETYP_RASTER,GEBURTSJAHR,GREEN_AVANTGARDE,HH_EINKOMMEN_SCORE,INNENSTADT,KBA05_ALTER1,KBA05_ALTER2,KBA05_ALTER3,KBA05_ALTER4,KBA05_ANHANG,KBA05_ANTG1,KBA05_ANTG2,KBA05_ANTG3,KBA05_ANTG4,KBA05_AUTOQUOT,KBA05_BAUMAX,KBA05_CCM1,KBA05_CCM2,KBA05_CCM3,KBA05_CCM4,KBA05_DIESEL,KBA05_FRAU,KBA05_GBZ,KBA05_HERST1,KBA05_HERST2,KBA05_HERST3,KBA05_HERST4,KBA05_HERST5,KBA05_HERSTTEMP,KBA05_KRSAQUOT,KBA05_KRSHERST1,KBA05_KRSHERST2,KBA05_KRSHERST3,KBA05_KRSKLEIN,KBA05_KRSOBER,KBA05_KRSVAN,KBA05_KRSZUL,KBA05_KW1,KBA05_KW2,KBA05_KW3,KBA05_MAXAH,KBA05_MAXBJ,KBA05_MAXHERST,KBA05_MAXSEG,KBA05_MAXVORB,KBA05_MOD1,KBA05_MOD2,KBA05_MOD3,KBA05_MOD4,KBA05_MOD8,KBA05_MODTEMP,KBA05_MOTOR,KBA05_MOTRAD,KBA05_SEG1,KBA05_SEG10,KBA05_SEG2,KBA05_SEG3,KBA05_SEG4,KBA05_SEG5,KBA05_SEG6,KBA05_SEG7,KBA05_SEG8,KBA05_SEG9,KBA05_VORB0,KBA05_VORB1,KBA05_VORB2,KBA05_ZUL1,KBA05_ZUL2,KBA05_ZUL3,KBA05_ZUL4,KBA13_ALTERHALTER_30,KBA13_ALTERHALTER_45,KBA13_ALTERHALTER_60,KBA13_ALTERHALTER_61,KBA13_ANZAHL_PKW,KBA13_AUDI,KBA13_AUTOQUOTE,KBA13_BJ_1999,KBA13_BJ_2000,KBA13_BJ_2004,KBA13_BJ_2006,KBA13_BJ_2008,KBA13_BJ_2009,KBA13_BMW,KBA13_CCM_0_1400,KBA13_CCM_1000,KBA13_CCM_1200,KBA13_CCM_1400,KBA13_CCM_1401_2500,KBA13_CCM_1500,KBA13_CCM_1600,KBA13_CCM_1800,KBA13_CCM_2000,KBA13_CCM_2500,KBA13_CCM_2501,KBA13_CCM_3000,KBA13_CCM_3001,KBA13_FAB_ASIEN,KBA13_FAB_SONSTIGE,KBA13_FIAT,KBA13_FORD,KBA13_HALTER_20,KBA13_HALTER_25,KBA13_HALTER_30,KBA13_HALTER_35,KBA13_HALTER_40,KBA13_HALTER_45,KBA13_HALTER_50,KBA13_HALTER_55,KBA13_HALTER_60,KBA13_HALTER_65,KBA13_HALTER_66,KBA13_HERST_ASIEN,KBA13_HERST_AUDI_VW,KBA13_HERST_BMW_BENZ,KBA13_HERST_EUROPA,KBA13_HERST_FORD_OPEL,KBA13_HERST_SONST,KBA13_KMH_0_140,KBA13_KMH_110,KBA13_KMH_140,KBA13_KMH_140_210,KBA13_KMH_180,KBA13_KMH_210,KBA13_KMH_211,KBA13_KMH_250,KBA13_KMH_251,KBA13_KRSAQUOT,KBA13_KRSHERST_AUDI_VW,KBA13_KRSHERST_BMW_BENZ,KBA13_KRSHERST_FORD_OPEL,KBA13_KRSSEG_KLEIN,KBA13_KRSSEG_OBER,KBA13_KRSSEG_VAN,KBA13_KRSZUL_NEU,KBA13_KW_0_60,KBA13_KW_110,KBA13_KW_120,KBA13_KW_121,KBA13_KW_30,KBA13_KW_40,KBA13_KW_50,KBA13_KW_60,KBA13_KW_61_120,KBA13_KW_70,KBA13_KW_80,KBA13_KW_90,KBA13_MAZDA,KBA13_MERCEDES,KBA13_MOTOR,KBA13_NISSAN,KBA13_OPEL,KBA13_PEUGEOT,KBA13_RENAULT,KBA13_SEG_GELAENDEWAGEN,KBA13_SEG_GROSSRAUMVANS,KBA13_SEG_KLEINST,KBA13_SEG_KLEINWAGEN,KBA13_SEG_KOMPAKTKLASSE,KBA13_SEG_MINIVANS,KBA13_SEG_MINIWAGEN,KBA13_SEG_MITTELKLASSE,KBA13_SEG_OBEREMITTELKLASSE,KBA13_SEG_OBERKLASSE,KBA13_SEG_SONSTIGE,KBA13_SEG_SPORTWAGEN,KBA13_SEG_UTILITIES,KBA13_SEG_VAN,KBA13_SEG_WOHNMOBILE,KBA13_SITZE_4,KBA13_SITZE_5,KBA13_SITZE_6,KBA13_TOYOTA,KBA13_VORB_0,KBA13_VORB_1,KBA13_VORB_1_2,KBA13_VORB_2,KBA13_VORB_3,KBA13_VW,KKK,KONSUMNAEHE,LP_FAMILIE_FEIN,LP_FAMILIE_GROB,LP_LEBENSPHASE_FEIN,LP_LEBE

In [ ]:
start = 0
step = 10000
end = step

for i in range(1, 81):
  print(f"split {i}: {start} - {end}")
  scaled_df.iloc[start:end,:].to_csv(f"df_azdias_scaled_{i}.csv", index=False)
  files = drive.CreateFile({'title' : f"df_azdias_scaled_{i}.csv"})
  files.SetContentFile(f"df_azdias_scaled_{i}.csv")
  files.Upload()
  drive.CreateFile({'id': files.get(f"df_azdias_scaled_{i}.csv")})
  start += step
  end += step
  print("____________________________")
print("upload completed")

# Part 1: Customer Segmentation Report

### PCA Analysis

In [4]:
# Get list of every available file in google drive
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
ticker=0
data=dict()
for file1 in file_list:
  if(ticker == 80):
    break
  else:
    data[file1['title']] = file1['id']
  ticker+=1

In [5]:
# Load data
download_data = drive.CreateFile({'id': data[f"df_azdias_scaled_{1}.csv"]})
download_data.GetContentFile(f"df_azdias_scaled_{1}.csv")  
df = pd.read_csv(f"df_azdias_scaled_{1}.csv")

for i in range(2, 81):
  if(i in [10, 20, 30, 40, 50, 60, 70, 80]):
    print(f"Slice {i}")
  download_data = drive.CreateFile({'id': data[f"df_azdias_scaled_{i}.csv"]})
  download_data.GetContentFile(f"df_azdias_scaled_{i}.csv")  
  temp = pd.read_csv(f"df_azdias_scaled_{i}.csv")

  df = pd.concat([df, temp], axis=0)

print(df.shape)
print(sum(df.isna().sum()))
df.head(3)

Slice 10
Slice 20
Slice 30
Slice 40
Slice 50
Slice 60
Slice 70
Slice 80
(785421, 385)
0


,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,BALLRAUM,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,D19_DIGIT_SERV,D19_DROGERIEARTIKEL,D19_ENERGIE,D19_FREIZEIT,D19_GARTEN,D19_GESAMT_ANZ_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_OFFLINE_DATUM,D19_GESAMT_ONLINE_DATUM,D19_GESAMT_ONLINE_QUOTE_12,D19_HANDWERK,D19_HAUS_DEKO,D19_KINDERARTIKEL,D19_KOSMETIK,D19_LEBENSMITTEL,D19_LOTTO,D19_NAHRUNGSERGAENZUNG,D19_RATGEBER,D19_REISEN,D19_SAMMELARTIKEL,D19_SCHUHE,D19_SONSTIGE,D19_TECHNIK,D19_TELKO_ANZ_12,D19_TELKO_ANZ_24,D19_TELKO_DATUM,D19_TELKO_MOBILE,D19_TELKO_OFFLINE_DATUM,D19_TELKO_ONLINE_DATUM,D19_TELKO_REST,D19_TIERARTIKEL,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,D19_VERSAND_DATUM,D19_VERSAND_OFFLINE_DATUM,D19_VERSAND_ONLINE_DATUM,D19_VERSAND_ONLINE_QUOTE_12,D19_VERSAND_REST,D19_VERSI_ANZ_12,D19_VERSI_ANZ_24,D19_VERSICHERUNGEN,D19_VOLLSORTIMENT,D19_WEIN_FEINKOST,EWDICHTE,FINANZ_ANLEGER,FINANZ_HAUSBAUER,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_UNAUFFAELLIGER,FINANZ_VORSORGER,GEBAEUDETYP_RASTER,GEBURTSJAHR,GREEN_AVANTGARDE,HH_EINKOMMEN_SCORE,INNENSTADT,KBA05_ALTER1,KBA05_ALTER2,KBA05_ALTER3,KBA05_ALTER4,KBA05_ANHANG,KBA05_ANTG1,KBA05_ANTG2,KBA05_ANTG3,KBA05_ANTG4,KBA05_AUTOQUOT,KBA05_BAUMAX,KBA05_CCM1,KBA05_CCM2,KBA05_CCM3,KBA05_CCM4,KBA05_DIESEL,KBA05_FRAU,KBA05_GBZ,KBA05_HERST1,KBA05_HERST2,KBA05_HERST3,KBA05_HERST4,KBA05_HERST5,KBA05_HERSTTEMP,KBA05_KRSAQUOT,KBA05_KRSHERST1,KBA05_KRSHERST2,KBA05_KRSHERST3,KBA05_KRSKLEIN,KBA05_KRSOBER,KBA05_KRSVAN,KBA05_KRSZUL,KBA05_KW1,KBA05_KW2,KBA05_KW3,KBA05_MAXAH,KBA05_MAXBJ,KBA05_MAXHERST,KBA05_MAXSEG,KBA05_MAXVORB,KBA05_MOD1,KBA05_MOD2,KBA05_MOD3,KBA05_MOD4,KBA05_MOD8,KBA05_MODTEMP,KBA05_MOTOR,KBA05_MOTRAD,KBA05_SEG1,KBA05_SEG10,KBA05_SEG2,KBA05_SEG3,KBA05_SEG4,KBA05_SEG5,KBA05_SEG6,KBA05_SEG7,KBA05_SEG8,KBA05_SEG9,KBA05_VORB0,KBA05_VORB1,KBA05_VORB2,KBA05_ZUL1,KBA05_ZUL2,KBA05_ZUL3,KBA05_ZUL4,KBA13_ALTERHALTER_30,KBA13_ALTERHALTER_45,KBA13_ALTERHALTER_60,KBA13_ALTERHALTER_61,KBA13_ANZAHL_PKW,KBA13_AUDI,KBA13_AUTOQUOTE,KBA13_BJ_1999,KBA13_BJ_2000,KBA13_BJ_2004,KBA13_BJ_2006,KBA13_BJ_2008,KBA13_BJ_2009,KBA13_BMW,KBA13_CCM_0_1400,KBA13_CCM_1000,KBA13_CCM_1200,KBA13_CCM_1400,KBA13_CCM_1401_2500,KBA13_CCM_1500,KBA13_CCM_1600,KBA13_CCM_1800,KBA13_CCM_2000,KBA13_CCM_2500,KBA13_CCM_2501,KBA13_CCM_3000,KBA13_CCM_3001,KBA13_FAB_ASIEN,KBA13_FAB_SONSTIGE,KBA13_FIAT,KBA13_FORD,KBA13_HALTER_20,KBA13_HALTER_25,KBA13_HALTER_30,KBA13_HALTER_35,KBA13_HALTER_40,KBA13_HALTER_45,KBA13_HALTER_50,KBA13_HALTER_55,KBA13_HALTER_60,KBA13_HALTER_65,KBA13_HALTER_66,KBA13_HERST_ASIEN,KBA13_HERST_AUDI_VW,KBA13_HERST_BMW_BENZ,KBA13_HERST_EUROPA,KBA13_HERST_FORD_OPEL,KBA13_HERST_SONST,KBA13_KMH_0_140,KBA13_KMH_110,KBA13_KMH_140,KBA13_KMH_140_210,KBA13_KMH_180,KBA13_KMH_210,KBA13_KMH_211,KBA13_KMH_250,KBA13_KMH_251,KBA13_KRSAQUOT,KBA13_KRSHERST_AUDI_VW,KBA13_KRSHERST_BMW_BENZ,KBA13_KRSHERST_FORD_OPEL,KBA13_KRSSEG_KLEIN,KBA13_KRSSEG_OBER,KBA13_KRSSEG_VAN,KBA13_KRSZUL_NEU,KBA13_KW_0_60,KBA13_KW_110,KBA13_KW_120,KBA13_KW_121,KBA13_KW_30,KBA13_KW_40,KBA13_KW_50,KBA13_KW_60,KBA13_KW_61_120,KBA13_KW_70,KBA13_KW_80,KBA13_KW_90,KBA13_MAZDA,KBA13_MERCEDES,KBA13_MOTOR,KBA13_NISSAN,KBA13_OPEL,KBA13_PEUGEOT,KBA13_RENAULT,KBA13_SEG_GELAENDEWAGEN,KBA13_SEG_GROSSRAUMVANS,KBA13_SEG_KLEINST,KBA13_SEG_KLEINWAGEN,KBA13_SEG_KOMPAKTKLASSE,KBA13_SEG_MINIVANS,KBA13_SEG_MINIWAGEN,KBA13_SEG_MITTELKLASSE,KBA13_SEG_OBEREMITTELKLASSE,KBA13_SEG_OBERKLASSE,KBA13_SEG_SONSTIGE,KBA13_SEG_SPORTWAGEN,KBA13_SEG_UTILITIES,KBA13_SEG_VAN,KBA13_SEG_WOHNMOBILE,KBA13_SITZE_4,KBA13_SITZE_5,KBA13_SITZE_6,KBA13_TOYOTA,KBA13_VORB_0,KBA13_VORB_1,KBA13_VORB_1_2,KBA13_VORB_2,KBA13_VORB_3,KBA13_VW,KKK,KONSUMNAEHE,LP_FAMILIE_FEIN,LP_FAMILIE_GROB,LP_LEBENSPHASE_FEIN,LP_LEBE

In [10]:
pca = PCA(.9)

In [11]:
pca.fit(np.array(df))

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [28]:
pca.n_features_

385

In [12]:
pca.n_components_

196

In [21]:
reduced_data = pca.transform(np.array(df))

In [30]:
reduced_data.shape

(785421, 196)

# Functions

In [ ]:
def engineer_CAMEO_INTL_2015(x):
  '''
  Transform value for feature CAMEO_INTL_2015.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if((x == 'X') | (x == 'XX')):
    return np.nan
  if(str(x)[0] == '1'):
    return '1'
  if(str(x)[0] == '2'):
    return '2'
  if(str(x)[0] == '3'):
    return '3'
  if(str(x)[0] == '4'):
    return '4'
  if(str(x)[0] == '5'):
    return '5'
  else:
    return np.nan

In [ ]:
def engineer_OST_WEST_KZ(x):
  '''
  Transform value for feature OST_WEST_KZ.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(x == 'W'):
    return 1
  elif(x == 'O'):
    return 0
  else:
    return np.nan

In [ ]:
def create_meta_dataframa():
  '''
  Creates data frame with meta information.

  Args:
  - Nothing

  Returns:
  - meta_df
  '''
  meta_df = pd.DataFrame(df_azdias.columns.tolist())
  meta_df = meta_df.rename(columns={0: "column_name"})
  meta_df["na_percentage_azdias"] = meta_df["column_name"].apply(lambda x: round(df_azdias[x].isna().sum()/df_azdias.shape[0], 4))
  meta_df["na_percentage_customers"] = meta_df["column_name"].apply(lambda x: round(df_customers[x].isna().sum()/df_customers.shape[0], 4))
  meta_df["dtypes_azdias"] =  meta_df["column_name"].apply(lambda x: set(df_azdias[x].map(type).tolist()))
  meta_df["dtypes_customers"] =  meta_df["column_name"].apply(lambda x: set(df_customers[x].map(type).tolist()))
  meta_df["n_dtypes_azdias"] = meta_df["column_name"].apply(lambda x: len(set(df_azdias[x].map(type).tolist())))
  meta_df["n_dtypes_customers"] = meta_df["column_name"].apply(lambda x: len(set(df_customers[x].map(type).tolist())))
  meta_df["unique_values_azdias"] = meta_df["column_name"].apply(lambda x: len(df_azdias[x].value_counts()))
  meta_df["unique_values_customers"] = meta_df["column_name"].apply(lambda x: len(df_customers[x].value_counts()))
  meta_df["description"] =  meta_df["column_name"].apply(get_description)
  meta_df["action"] = "Keep"
  meta_df["reason"] = "None"
  return meta_df

In [ ]:
def engineer_PRAEGENDE_JUGENDJAHRE(x):
  '''
    Transform value for feature PRAEGENDE_JUGENDJAHRE.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(x in [1, 2]):
    return 40
  elif(x in [3, 4]):
    return 50
  elif(x in [5, 6, 7]):
    return 60
  elif(x in [8, 9]):
    return 70
  elif(x in [10, 11, 12, 13]):
    return 80
  elif(x in [14, 15]):
    return 90
  else:
    return np.nan

In [ ]:
def engineer_ALTER_HH(x):
  '''
  Transform value for feature ALTER_HH.

  Args:
  - x: Cell value

  Returns:
  - Transformed x
  '''
  if(x in [1, 2, 3]):
    return 10
  elif(x in [3, 4, 5]):
    return 20
  elif(x in [6, 7]):
    return 30
  elif(x in [8, 9]):
    return 40
  elif(x in [10, 11]):
    return 50
  elif(x in [12, 13]):
    return 60
  elif(x in [14, 15]):
    return 70
  elif(x in [16, 17]):
    return 80
  elif(x in [18, 19]):
    return 90
  elif(x in [20, 21]):
    return 100
  else:
    return np.nan
